In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import requests
import re
import time
from time import sleep
            

# 크롬 드라이버 다운로드 및 자동 설정
chrome_driver_path = ChromeDriverManager().install()

# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 삭제
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬 브라우저를 열고 cbs news 웹페이지로 이동
region = '광진구음식점'
url = f"https://map.naver.com/p/search/{region}"
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=chrome_options)
driver.get(url)
# driver.maximize_window()
time.sleep(10)

In [47]:
# iframe으로 전환
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
driver.switch_to.frame(iframe)
time.sleep(3)
print("iframe 전환")

iframe 전환


In [11]:
def scroll_down():
    # 한 번에 10개의 식당이 나오고, 스크롤 시 동적으로 추가되어 총 50개까지 한 페이지 안에 나온다
    for i in range(4):
        # 페이지 아래로 스크롤
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container"))
        time.sleep(3)
    


In [12]:
def next_page():

    # '>' 버튼 찾기
    next_button = driver.find_elements(By.CSS_SELECTOR, ".eUTV2")[-1]
    if next_button.get_attribute("aria-disabled") == 'true':
        print("Last Page")
        return -1

    else:
        next_button.click()
        print("click")

    time.sleep(3)

# for i in range(4):
#     tag = next_page()
#     if tag == -1:
#         break


In [ ]:
# driver.switch_to.default_content()

from selenium.webdriver.common.by import By

# 현재 프레임을 변경한 적이 있는지 여부를 확인
try:
    driver.find_element(By.XPATH, ".//*")
    print("현재 프레임: 최상위 컨텍스트")
except:
    # 만약 예외가 발생한다면 현재 프레임이 설정된 것임
    print("현재 프레임:", driver.current_frame)


In [ ]:
driver.switch_to.default_content()
# iframe 나가기

In [ ]:
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
rst_taps = driver.switch_to.frame(iframe)

# 레스토랑 탭 들어가기

In [40]:
# 식당정보 수집
def crawl_rst_info():
    try:
        rst_name = driver.find_element(By.CSS_SELECTOR, ".Fc1rA").text              # 식당이름
    except:
        rst_name = None
    try:
        rst_category =  driver.find_element(By.CSS_SELECTOR, ".DJJvD").text         # 식당 카테고리
    except:
        rst_category = None
    try:
        rst_nreviews =  driver.find_elements(By.CSS_SELECTOR, ".PXMot")[-2].text    # 식당 리뷰 수
    except:
        rst_nreviews = None
    try:
        rst_address =  driver.find_element(By.CSS_SELECTOR, ".LDgIH").text          # 식당 주소
    except:
        rst_address = None
    try:
        rst_number = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text            # 식당 전화번호
    except:
        rst_number = None

    time_button = driver.find_elements(By.CSS_SELECTOR, "._UCia")[1].click()        # 식당 시간 더보기 버튼 클릭
    time.sleep(2)
    rst_times =  driver.find_elements(By.CSS_SELECTOR, ".A_cdD")[1:]                # 요일별 영업시간들
    times = []
    for rst_time in rst_times:
        times.append(rst_time.text)



    tabs = driver.find_elements(By.CSS_SELECTOR, "._tab-menu")
    for tab in tabs:
        if tab.text == '메뉴':
            menu_button = tab                                                       # 메뉴 정보 수집

    menu_button.click()
    time.sleep(1)
    menus_tab = driver.find_elements(By.CSS_SELECTOR, ".E2jtL")
    menus = {}
    for menu in menus_tab:
        title = menu.find_element(By.CSS_SELECTOR, ".lPzHi").text
        price = menu.find_element(By.CSS_SELECTOR, ".GXS1X")
        price = price.find_element(By.TAG_NAME, "em").text                          # 가격 정보 함께 수집

        menus[title] = price


    print(rst_name)       # clear
    print(rst_category)   # clear
    print(rst_nreviews)   # clear
    print(rst_address)    # clear
    print(rst_number)     # clear
    print(times)          # clear
    print(menus)          # clear

    rst_info = [rst_name, rst_category, rst_address, times, rst_nreviews, rst_number, menus, 0]

    return rst_info

In [48]:
rst_test = driver.find_element(By.CSS_SELECTOR, ".UEzoS")
rst = rst_test.find_element(By.CSS_SELECTOR, ".tzwk0")

# 레스토랑 하나 테스트 용으로 방문
rst.click()
time.sleep(1)

# 전체 페이지로 복귀하도록 iframe에서 벗어나기 -> 그래야만 새로 생긴 iframe 탭에 접근할 수 있음
driver.switch_to.default_content()

# 새로 생긴 식당 탭 iframe으로 전환
iframe = driver.find_element(By.CSS_SELECTOR, "#entryIframe")
driver.switch_to.frame(iframe)
time.sleep(1)
# print("식당 탭 iframe 전환")

"""
    
정보 수집
"""

info = crawl_rst_info()

# menu_button = driver.find_elements(By.CSS_SELECTOR, "._tab-menu")[1].click()    # 메뉴 정보 수집
# time.sleep(1)

# menus_tab = driver.find_elements(By.CSS_SELECTOR, ".E2jtL")
# menus = {}
# for menu in menus_tab:
#     title = menu.find_element(By.CSS_SELECTOR, ".lPzHi").text
#     price = menu.find_element(By.CSS_SELECTOR, ".GXS1X")
#     price = price.find_element(By.TAG_NAME, "em").text

#     menus[title] = price

# print(menus)
# time_button = driver.find_elements(By.CSS_SELECTOR, "._UCia")[1].click()           # 식당 시간 더보기 버튼 클릭
# rst_days =  driver.find_elements(By.CSS_SELECTOR, ".i8cJw")        # 영업 요일
# rst_time =  driver.find_elements(By.CSS_SELECTOR, ".H3ua4")        # 영업 요일별 시간

# xxx = driver.find_elements(By.CSS_SELECTOR, ".A_cdD")[1:]

# times = []

# for x in xxx:
#     print(x.text)


이자카야청심
이자카야
방문자리뷰 173
서울 광진구 동일로22길 37 1층 이자카야 청심
0507-1392-0593
['매일\n17:00 - 04:00']
{'해물야끼소바': '18,000', '시메사바': '13,000', '숯불시사모구이': '15,000', '오늘의꼬치7종세트': '18,000'}


In [ ]:
rsts = driver.find_elements(By.CSS_SELECTOR, ".UEzoS")

for rst in rsts:
    rst = rst.find_element(By.CSS_SELECTOR, ".tzwk0")

    # 레스토랑 방문
    rst.click()
    time.sleep(2)

    # 전체 페이지로 복귀하도록 iframe에서 벗어나기 -> 그래야만 새로 생긴 iframe 탭에 접근할 수 있음
    driver.switch_to.default_content()

    # 새로 생긴 식당 탭 iframe으로 전환
    iframe = driver.find_element(By.CSS_SELECTOR, "#entryIframe")
    driver.switch_to.frame(iframe)
    time.sleep(2)
    print("식당 탭 iframe 전환")

    """
    
    정보 수집
    """
    crawl_rst_info()

    # iframe 다시 레스토랑 전체 탭으로 복귀
    driver.switch_to.default_content()
    iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
    driver.switch_to.frame(iframe)
    time.sleep(2)
    

In [78]:
rst_time = driver.find_elements(By.CSS_SELECTOR, ".A_cdD")
for time in rst_time:
    print(time.text)

In [77]:
rst_time

[]

In [50]:
print(rst_name)
print(rst_category)
print(rst_nreviews)   # clear
print(rst_address)    # clear
print(times)

불야성
요리주점
서울 광진구 동일로22길 89 1층
방문자리뷰 87
['영업 중\n08:00에 영업 종료\n8시 0분에 영업 종료', '매일\n17:00 - 08:00']


In [22]:
for time in rst_time:
    print(time.text)

영업 중
08:00에 영업 종료
8시 0분에 영업 종료
매일
17:00 - 08:00


In [1]:
name = 'abc'
number = 123
a = [name, number]

In [2]:
a

['abc', 123]